<div class="alert alert-block alert-warning">
<h1><span style="color:green"> Under-Graduate Research Internship Program (UGRIP) - 2024 <br> Lab 01 - Part B </span><h1>

<h2><span style="color:green"> Classification Algorithms: K-Nearest Neighbors (KNN), SVMs, Decision Trees and Random Forests</span><h2>
</div>

In this session we're going to take a closer look at specialized classification algorithms to understand their behavior and investigate their properties.

# 1. k-Nearest Neighbors (kNN)

k-Nearest Neighbors is a simple supervised machine learning algorithm which can be used for both classification and regression tasks. Unlike most other estimators, it doesn't attempt to fit or learn any model parameters i.e. it is model-free. Rather, it stores the training data internally (formally they are called "exemplars"). That's right; no training involved.

When you want to use it on new data (formally called "inference time") and you present it with new data, it simply compares the new data point to all the exemplars it has stored. Once the "nearest neighbor" of this unknown point has been located, then the new point takes on the label of this nearest neighbor.

Now, this classifier has only one "hyperparameter" (a hyperparameter is a user-specified number that controls the operation of the estimator/model, but is not learnt from data), which is $k$; the number of neighbors involved in the labelling process. In our explanation above, we consider only the (one) nearest neighbor, so that is the operation of a 1-Nearest Neighbor classifier. We can also use more neighbors e.g. $k=3$, in which case the label of the new, unseen point will be the majority vote of the labels of the 3 nearest neighbors and so on.

As usual, sklearn comes with an implementation of the k-Nearest Neighbor classifier in the `sklearn.neighbors` module. Let us import it and other relevant imports below:

In [ ]:
import numpy as np                 # linear algebra and numerical computing
import matplotlib.pyplot as plt    # for data visualization 

from sklearn.neighbors import KNeighborsClassifier

Now, let us consider the famous Iris dataset. This dataset consists of 4 measurements taken for several Iris plants which belong to 3 species. We will use this dataset to explore the properties/behavior of the kNN. 

Go ahead and load the data from the `sklearn.datasets` module using the `load_iris()` function.

In [ ]:
from sklearn.datasets import load_iris

# Implement Here

Now, go ahead to separate the data into features and targets. <ins> Select the first two features in the dataset</ins>, and make a training and test split (75%/25% ratio). Set the random seed to 1000 for reproducibility (by setting the `random_state` argument to 1000).

In [ ]:
# Implement Here
features, labels = ..., ...

Finally, go ahead fit a 1-Nearest Neighbor classifier to it and evaluate the performance of the classifier using the accuracy evaluation metric:

In [ ]:
# Implement Here

What do you think of the k-NN performance? Do you think this is a good classifier?

## Effect of $k$

Now, you will do an experiment to explore the effect of $k$ on the classification accuracy of the classifier. You need to do the following:
- Consider values from $k$ = 1 to 11
    - For each $k$ value, initialize and fit a kNN with the number of neighbors set to the current $k$ value
    - Compute the accuracy at this value of $k$

When finished, plot the classifier accuracy as a function of $k$. This is just a fancy way of saying put the value of $k$ on the x-axis, and the corresponding classifier accuracy on the y-axis. Bonus points for nice visual presentation of results.


In [ ]:
# Implement Here

We will revisit this phenomenon shortly.

## Effect of Scaling on the k-NN

Let us try a simple experiment. What happens if we multiply one feature to make it much larger than the other? Would the k-NN still work well?
To investigate this, we will multiply one feature by 1000, then split the data again:

In [ ]:
features[:, 0] *= 1000

Then let repeat the data splitting process (using our fixed random seed for reproducibility):

In [ ]:
X_train, X_test, Y_train, Y_test = ...

Finally, repeat the plot we made above:

In [ ]:
# Implement Here

As you can see, this seems to have harmed the kNN severely. This is because of how it works, as it relies on something called a distance function.

## Distance Functions

Recall that a kNN assigns a new point to the label of its nearest exemplar(s). But how can we determine the concept of nearness?
From an ML perspective, we need to have a way to measure the nearness or closeness of two points i.e. to put a number on it. We can rephrase that and say that we need a way to measure _distance_ between the two points. 

This latter formulation is nicer because it has better properties both intuitively and mathematically. For instance, if two points have a distance of 0 between them, then they are the same point. If we had used the other definition, then it would be hard to put a hard number of the degree of closeness of two points.

Therefore, this motivates a distance function. A distance function takes two points (or more concretely, the numbers/features specifying two points), and returns a quantitative measure of the distance between them. For instance, you have dealt with Euclidean distance in your math courses, which says that the distance between two points $P_1 = (x_1, y_1)$ and $P_2 = (x_2, y_2)$ in Euclidean space is given by:
$$
    D(P_1, P_2) = \sqrt{(x_2 - x_1)^2 + (y_2 - y_1)^2}
$$



Incidentally, the kNN relies on the exact same Euclidean distance function for its operation. 

Now that you know this, can you guess why increasing the magnitude of one of the features negatively affected its operation? More importantly, can you suggest potential ways to fix the problem?

There are many other distance functions, each of which have distinct properties. For example, cosine distance is one type of distance function which measures the _angle_ between two points (actually two vectors with a common point of reference). For this reason, cosine distance is what is termed as being _scale-invariant_. 

Your next task is to prove this by:
- reloading the iris data and selecting its first two columns/features again
- fitting a 1-NN (using cosine distance) to it and computing its accuracy
- multiplying one of the features of the data, and refitting the 1-NN to this rescaled data
- and recomputing the accuracy
- finally, compare the two accuracies

In [ ]:
# Implement Here

What do you observe?

# Decision Boundaries

You can think of a classifier as basically trying to find a way to separate the items from different classes. For instance, consider a simple case where you have data belonging to some classes in a 2D space (i.e. having two features, so you can plot them on a sheet of paper). We can even consider our dataset from above accordingly:

In [ ]:
iris = load_iris()
features, labels = iris.data[:, :2], iris.target
plt.scatter(features[:, 0], features[:,1], c=labels)
plt.show()

Can you draw a line that perfectly separates them?

All classifiers basically try to achieve that goal using principled means. For instance, logistic regression (which is a linear classifier) tries to find lines to separate the data from each other by adjusting its weights. The kNN on the other hand doesn't explicitly try to find a line, but instead stumbles upon one or induces one based on the data distribution. 

It would be nice to be able to see how each classifier goes about drawing these separating lines (which are termed "Decision Boundaries"), as that will tell us what the classifier is thinking.

In 2D (such as in our case here), it is fairly easy to do this. Sklearn comes with functionality to draw the decision boundary of a classifier by using the `DecisionBoundaryDisplay` class (from the `sklearn.inspection` module). It is fairly complex to use, but we can easily create one using its `from_estimator()` function, which we will then use.

To kill two birds with one stone, let us now create two kNNs: one with $k = 1$ and the other with $k = 9$ (from our graph earlier), and see the differences in their behavior via their decision boundary plots:

In [ ]:
from sklearn.inspection import DecisionBoundaryDisplay

X_train, X_test, Y_train, Y_test = train_test_split(features, labels, train_size=0.75, random_state=1000)
knn_1 = KNeighborsClassifier(1).fit(X_train, Y_train)
knn_11 = KNeighborsClassifier(9).fit(X_train, Y_train)

predictions_1 = knn_1.predict(X_test)
predictions_11 = knn_11.predict(X_test)

fig, axes = plt.subplots(1,2)

disp1 = DecisionBoundaryDisplay.from_estimator(
    knn_1, X_train, response_method="predict",
    alpha=0.5,
    ax=axes[0]
)
axes[0].scatter(X_train[:, 0], X_train[:, 1], c=Y_train, edgecolor="k")

disp1 = DecisionBoundaryDisplay.from_estimator(
    knn_11, X_train, response_method="predict",
    alpha=0.5,
    ax=axes[1]
)
axes[1].scatter(X_train[:, 0], X_train[:, 1], c=Y_train, edgecolor="k")

plt.show()

### Weaknesses of the kNN

Now that we have seen and experimented with the kNN, can you come up with its weaknesses? You get one for free i.e. sensitive to feature scales. But are there others you can think of?

---

# 2. Support Vector Machines

Support vector machines (or SVMs) are another very popular algorithm for classification (although they can also be used for regression as well). Unlike the other classifiers we've seen, they take a completely different approach while taking some of the limitations we've seen so far into consideration e.g. linear assumptions.

Rather than worry about all the training data, SVMs instead focus on making decision boundaries that are maximally separated from the borders of the two classes. In other words, they focus only on the training data points that are closest to the decision boundary. It is these "extreme" points that are called Support Vectors. 

## Kernel Functions

SVMs also realize that it is sometimes hard to linearly separate the data classes because sometimes the boundaries can be irregular. As a result, SVMs allow for the use of data projection functions internally. The assumption is: if the data cannot be linearly separated in its native/natural space, then maybe it can be more easily separated in a higher dimensional space. Intuitively, think about it like this: it might be hard to distinguish between two people (say twins to make it hard) based on their heights and weights alone (since they are physically similar). But if we add new features e.g. the favorite animes of each twin, the names of their best friends, etc, then they will be easier to distinguish. 

SVMs achieve this by: taking the data points and passing them through a transformation function to increase their dimensionality, then trying to separate them in this new high dimensional form by estimating similarity in a kernel-specific way. For simplicity we can call these special projection + similarity-measuring functions "kernel functions". The exact kernel function used is set by the user i.e. it is a hyperparameter, not a learned parameter. We will explore the different kernel functions SVMs offer to see their differences in the following sections.

## A Simple Experiment with Kernels

But first, let us try a small experiment with the SVM and our Iris classification problem from above. We will create four SVM classifiers, each one with a different kernel, fit them to the same data (`X_train` and `Y_train` from above) and compute their accuracy on the same test data (`X_test` and `Y_test`). This will give us the comparative performance at a glance of each of these kernels relative to each other.

To keep the code clean, we will use the `score()` function of each estimator. Every estimator - whether classification or regression - implements a `score()` function, which computes a default evaluation metric on the supplied data. For regression models, you get the R^2 score which measures correlation*. For classifiers, you get accuracy. Using this function we can cut down on code a little bit and get accuracy quickly.

In [ ]:
from sklearn.svm import SVC

a = SVC(kernel='linear').fit(X_train, Y_train).score(X_test, Y_test)
b = SVC(kernel='rbf').fit(X_train, Y_train).score(X_test, Y_test)
c = SVC(kernel='sigmoid').fit(X_train, Y_train).score(X_test, Y_test)
d = SVC(kernel='poly').fit(X_train, Y_train).score(X_test, Y_test)

print(a,b,c,d)

As you can see, the SVM easily rivals the k-NN with very little effort. 

Now, let us dig a bit deeper into these kernel functions. Consider the toy dataset below:

In [ ]:
X = np.array(
    [
        [0.4, -0.7],
        [-1.5, -1.0],
        [-1.4, -0.9],
        [-1.3, -1.2],
        [-1.1, -0.2],
        [-1.2, -0.4],
        [-0.5, 1.2],
        [-1.5, 2.1],
        [1.0, 1.0],
        [1.3, 0.8],
        [1.2, 0.5],
        [0.2, -2.0],
        [0.5, -2.4],
        [0.2, -2.3],
        [0.0, -2.7],
        [1.3, 2.1],
    ]
)

y = np.array([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1])
plt.rcParams["figure.figsize"] = plt.rcParamsDefault["figure.figsize"]
plt.scatter(X[:, 0], X[:, 1], s=150, c=y, edgecolors="k")
plt.title("Samples in two-dimensional feature space")
plt.show()

Similar to before, we will look at the decision boundary of the SVM. But what we will do is to swap out different kernels and see how the choice of kernel affects the shape of the decision boundary. For simplicity, we will define a function that accepts our target kernel as input and produces the corresponding decision boundary as output:

In [ ]:
from sklearn import svm
from sklearn.inspection import DecisionBoundaryDisplay


def plot_training_data_with_decision_boundary(
    kernel, ax=None, long_title=True, support_vectors=True
):
    # Train the SVC
    clf = svm.SVC(kernel=kernel, gamma=2).fit(X, y)

    # Settings for plotting
    if ax is None:
        _, ax = plt.subplots(figsize=(4, 3))
    x_min, x_max, y_min, y_max = -3, 3, -3, 3
    ax.set(xlim=(x_min, x_max), ylim=(y_min, y_max))

    # Plot decision boundary and margins
    common_params = {"estimator": clf, "X": X, "ax": ax}
    DecisionBoundaryDisplay.from_estimator(
        **common_params,
        response_method="predict",
        plot_method="pcolormesh",
        alpha=0.3,
    )
    DecisionBoundaryDisplay.from_estimator(
        **common_params,
        response_method="decision_function",
        plot_method="contour",
        levels=[-1, 0, 1],
        colors=["k", "k", "k"],
        linestyles=["--", "-", "--"],
    )

    if support_vectors:
        # Plot bigger circles around samples that serve as support vectors
        ax.scatter(
            clf.support_vectors_[:, 0],
            clf.support_vectors_[:, 1],
            s=150,
            facecolors="none",
            edgecolors="k",
        )

    # Plot samples by color and add legend
    scatter = ax.scatter(X[:, 0], X[:, 1], c=y, s=30, edgecolors="k")
    ax.legend(*scatter.legend_elements(), loc="upper right", title="Classes")
    if long_title:
        ax.set_title(f" Decision boundaries of {kernel} kernel in SVC")
    else:
        ax.set_title(kernel)

    if ax is None:
        plt.show()

With that done, we can now explore each kernel separately.

### Linear Kernel

In [ ]:
plot_training_data_with_decision_boundary("linear")

The linear kernel basically uses the raw feature space as-is i.e. doesn't actually involve any projection. As you can see, it draws a straight line that isn't any different than what anyone else might have drawn.

### Poly(nomial) Kernel

In [ ]:
plot_training_data_with_decision_boundary("poly")

As you can see, the polynomial kernel is able to come up with a flexible decision boundary. This is because it measures similarity differently than the linear kernel.

### RBF Kernel

In [ ]:
plot_training_data_with_decision_boundary("rbf")

The RBF kernel comes with decision boundaries that are shaped to the distribution of the data. This makes it very nice for irregular data distributions (which are very common in practice). Furthermore, it works equally well for well-behaved data. Because of its performance it is actually the default kernel for SVMs in sklearn.

### Sigmoid Kernel

In [ ]:
plot_training_data_with_decision_boundary("sigmoid")

The sigmoid kernel is closely related to the linear kernel. What it tries to do is to form an S-shaped decision boundary, which naturally only works if the data can fit in such a specific envelope. Because of this weakness, the sigmoid kernel is very rarely used in practice.

## The XOR Problem

There is a common problem in the ML community called the XOR problem, which has to do with a problem that is not linearly separable (recall that linear separability simply means you can divide the classes from each other using a straight line). Let us see how well the SVM performs on this problem with different kernels:

In [ ]:
xx, yy = np.meshgrid(np.linspace(-3, 3, 500), np.linspace(-3, 3, 500))
np.random.seed(0)
X = np.random.randn(300, 2)
y = np.logical_xor(X[:, 0] > 0, X[:, 1] > 0)

_, ax = plt.subplots(2, 2, figsize=(8, 8))
args = dict(long_title=False, support_vectors=False)
plot_training_data_with_decision_boundary("linear", ax[0, 0], **args)
plot_training_data_with_decision_boundary("poly", ax[0, 1], **args)
plot_training_data_with_decision_boundary("rbf", ax[1, 0], **args)
plot_training_data_with_decision_boundary("sigmoid", ax[1, 1], **args)
plt.show()

What are your thoughts? Which kernel do you think does best? Can you justify why?

We can even go a step further and plot the decision boundary of an RBF-SVM on our original Iris classification task. This is left as a quick exercise to you:

In [ ]:
# Implement Here

# 3. Decision Trees and Random Forests

Now we will explore our last classifier of this session. Decision trees are another type of classifier which take a different approach to classification. Rather than try to find a line or "optimal" decision boundary, they instead tackle the classification in a sequential manner. 

Say for example, you have a problem with X features. The Decision Tree (DT) classifier will randomly select one feature and then find a threshold that best separates the data into two groups e.g positive and negative. If you think of this graphically, data comes into a node, then there are two branches or routes coming out of it. Then for each (new) branch, the DT classifier randomly selects another feature to separate the data in that branch into two sets, and so on and so forth (recursive partitioning).

As the depth of the tree grows, the data at each node becomes more and more pure/homogenous. Eventually, the DT stops growing new nodes and branches when it is able to perfectly isolate a particular class in a branch (a terminal branch is technically called a leaf) or it reaches the maximum preset depth (a hyperparameter).

As you can see, DTs can be thought of in visual terms very easily. This makes them very easy to interpret (since you can draw what the tree is doing at each step in sequence, and they mimic human reasoning in the way they work), as we will see shortly.

## A Simple Example

Now, let us begin by applying a decision tree to our Iris classification problem above, both to see its performance and also to look at the sort of decision boundary it creates. We will use the DT implementation in sklearn (under the `sklearn.tree` module):

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier(criterion='entropy')
dtc.fit(X_train, Y_train)
print(dtc.score(X_test, Y_test))

As you can see, the DT works better than the kNN, but doesn't beat the SVM. However, it is interpretable, so we can examine its operation by visualizing the tree's nodes. To do that, we simply use the `plot_tree()` function in the `sklearn.tree` module:

In [ ]:
from sklearn.tree import plot_tree

plot_tree(dtc, filled=True)
plt.savefig('tree.png', dpi=500) #uncomment to save a figure that might be easier to see
plt.show()

We can also look at its decision boundary to compare with the others:

In [ ]:
predictions = dtc.predict(X_test)

DecisionBoundaryDisplay.from_estimator(
    dtc, X_train, response_method="predict",
    alpha=0.5,
)

plt.scatter(X_train[:, 0], X_train[:, 1], c=Y_train, edgecolor="k")
plt.show()

You will quickly observe that the DT makes decision boundaries that are axis-aligned i.e. parallel to either the x- or y-axis. Based on the way it works i.e. finding thresholds per feature, can you intuit why this is the case?

### Weaknesses of the DT

You may have noticed that the DT, even for our fairly simple 2D problem, made a fairly complex tree. This is because it is trying very hard to perfectly sort/classify the training data perfectly. While this might seem like a good idea, it turns out that it makes the DT fairly weak. This is because the DT basically has a tendency to be perfect for the training data, but as a result, be unable to work well for new, unseen data (because the thresholds it learned were meant for the training data and not necessarily for new, unseen data).
This phenomenon is called "overfitting", and will be explored in the last session for today.

Furthermore, depending on the initial feature selected, you can end up with vastly different trees (with vastly different performances), which definitely isn't very good news.

Wouldn't it be nice if we could somehow deal with these two problems? Enter the Random Forest Classifier!

## Random Forests

As the name suggests, a Random Forest (RF) classifier is a forest - made up of many individual trees. Random Forests were created to tackle the problems with DTs. 

If you are familiar with the phrase that "two heads are better than one", the Random Forests classifier embodies this reasoning. Rather than relying on a single tree (which can overfit), it incorporates multiple trees, each of them grown differently/independently of the other. To make the constituent trees even more diverse, Random Forests carry out something called Bootstrap Aggregation (also called Bagging). Basically, they randomly sample the training data, then grow each decision tree within them (the number is a hyperparameter) on a different random sample of the data. Therefore each decision tree sees a different selection of the training data, and individually selects its own set of features to split on, making each of the trees very different from each other. This allows for better diversity of opinions and is very useful in decision making (see below).

When it is time to take decisions on new data (i.e. at inference time), the Random Forest classifier allows each DT inside it to make its own decision, then takes a majority vote across all its DTs. This is a powerful idea because it means that even some of the individual DTs overfit, the ones that do not can somehow compensate in the decision making process, which makes the RF classifier generally much less prone to overfitting. Furthermore, by relying on multiple trees, we can eliminate the "flakiness" of the overall classifier since the RF is listening to many trees not just one potentially flaky one. This is how the RF solves the problems of the DT classifier.

PS: The sklearn implementation of RFs is found in `sklearn.ensemble.RandomForestClassifier`. Fitting it means fitting the DTs inside it, which will definitely take some time. But since each DT is independent of the other, they can be fit in parallel. This is achieved by setting the parameter `n_jobs=-1` when you create the RF classifier object.

PPS: A Random Forest classifier is called an Ensemble classifier because it is a big classifier made out of many smaller classifiers working together.

You can actually also plot the constituents of the RF classifier, but the figure will be quite complex and large so we will avoid doing that. Rather, we will try an algorithm shootout.

# 4. The Great Algorithm Shootout

Now, we are going to evaluate the classifier algorithms we have seen so far on a complex problem, in order to see how well each of them perform. You will be responsible for implementing this task in code and recording the results.

Your candidates are:
- 1-NN
- SVMs: RBF
- DT
- RF (300 trees)

Your target task is classification on the UJIIndoorLoc dataset (details [here](https://archive.ics.uci.edu/dataset/310/ujiindoorloc)). This is a very large dataset which consists of WiFi signal strengths and the corresponding floor/building they were recorded in. Your job is to test each of these classifiers on this task, reporting both their training and testing accuracy.

In this case, the dataset has already been pre-split into training (trainingData.csv) and testing (validationData.csv) portions. But to make things more interesting, we are going to train on the testing data, and test on the training data.

The features of interest are in columns 0 to 519, and the target labels are in column 522.

Hint: It might be a good idea to write a function to do this, since the pipeline is the same for all the classifiers. All that changes is the _type_ of classifier.

In [ ]:
# Implement Here

What can you observe from the results?